# Obtian the mastersheet for MrOs visit 2

## Visualize mastersheet visit 1

In [3]:
from pathlib import Path
import pandas as pd

dataset_path = Path("/Users/alicealbrecht/Desktop/UCSF/datasets")
mastersheet_visit1 = pd.read_csv(dataset_path / 
                                 "mastersheets" / 
                                 "mastersheet_MrOS_SOF_visit1.csv")
mastersheet_visit1.head()

,SID,Site,Dataset,Fs,StartTime,Channel,EDFPath,AnnotPath
0,bi0001,bi,MrOS,512.0,2004-09-12 20:16:58,"['Position', 'Leg L', 'Leg R', 'C3', 'C4', 'A1...",/data/haoqisun/dataset_MrOS_SOF_PSG/MrOS/bi/bi...,/data/haoqisun/dataset_MrOS_SOF_PSG/MrOS/bi/bi...
1,bi0002,bi,MrOS,512.0,2004-04-12 20:48:27,"['Position', 'Leg L', 'Leg R', 'C3', 'C4', 'A1...",/data/haoqisun/dataset_MrOS_SOF_PSG/MrOS/bi/bi...,/data/haoqisun/dataset_MrOS_SOF_PSG/MrOS/bi/bi...
2,bi0003,bi,MrOS,512.0,2004-02-24 20:00:25,"['Position', 'Leg L', 'Leg R', 'C3', 'C4', 'A1...",/data/haoqisun/dataset_MrOS_SOF_PSG/MrOS/bi/bi...,/data/haoqisun/dataset_MrOS_SOF_PSG/MrOS/bi/bi...
3,bi0004,bi,MrOS,512.0,2004-11-17 18:50:48,"['Position', 'Leg L', 'Leg R', 'C3', 'C4', 'A1...",/data/haoqisun/dataset_MrOS_SOF_PSG/MrOS/bi/bi...,/data/haoqisun/dataset_MrOS_SOF_PSG/MrOS/bi/bi...
4,bi0007,bi,MrOS,512.0,2004-09-02 18:18:03,"['Position', 'Leg L', 'Leg R', 'C3', 'C4', 'A1...",/data/haoqisun/dataset_MrOS_SOF_PSG/MrOS/bi/bi...,/data/haoqisun/dataset_MrOS_SOF_PSG/MrOS/bi/bi...


## One Subject

In [8]:
import pyedflib # ther option: mne
from pathlib import Path
import pandas as pd

# Load EDF
dataset_path = Path("/Users/alicealbrecht/Desktop/UCSF/datasets")
edf_path = dataset_path / "raw/MrOs/visit2/EDF/bi0010_20110717_EDF/bi0010_20110717.edf"

edf_file = pyedflib.EdfReader(str(edf_path))

# Get ECG sampling frequency
for i in range(edf_file.signals_in_file):
    label = edf_file.getLabel(i)
    if 'ECG' in label.upper(): 
        sfreq = edf_file.getSampleFrequency(i)
        break  # Stop after the first ECG channel
print("Fs:", sfreq)

# Get start date/time
start_date = edf_file.getStartdatetime()
print("StartTime:", start_date)


# Get channel names
channel_names = edf_file.getSignalLabels()
print("Channel:", channel_names)

# Close EDF
edf_file.close()

# Get paths
path = (Path.cwd() / "../data/MrOs_visit2/EDF/bi0010_20110717_EDF/").resolve()
edf_path = path / "bi0010_20110717.edf"
annot_path = path / "bi0010_20110717.edf.xml"
print("EDFPath:", edf_path)
print("AnnotPath:", annot_path)

Fs: 512.0
StartTime: 2011-07-17 18:35:12
Channel: ['Position', 'LegL', 'LegR', 'C3', 'C4', 'M1', 'M2', 'E2', 'E1', 'ECGL', 'ECGR', 'LChin', 'RChin', 'Airflow', 'SUM', 'Chest', 'ABD', 'STAT', 'HR', 'SpO2', 'CannulaFlow', 'DHR']
EDFPath: /Users/alicealbrecht/Documents/GitHub/PSG-Pipeline/data/MrOs_visit2/EDF/bi0010_20110717_EDF/bi0010_20110717.edf
AnnotPath: /Users/alicealbrecht/Documents/GitHub/PSG-Pipeline/data/MrOs_visit2/EDF/bi0010_20110717_EDF/bi0010_20110717.edf.xml


## Multiple subjects

In [3]:
import pyedflib
from pathlib import Path
import pandas as pd
from datetime import datetime

# Base EDF directory (contains subfolders for each subject)
PROJECT_ROOT = Path.cwd().resolve().parent # adjust level as needed
base_dir = PROJECT_ROOT / "datasets/raw/MrOS/visit2/EDF"
records = []

for subject_folder in base_dir.iterdir():
    if subject_folder.is_dir():
        folder_name = subject_folder.name
        sid = folder_name.split("_")[0]
        site = sid[:2]
        dataset = "MrOS"
        visit = "V2"

        all_files = list(subject_folder.iterdir())
        edf_path = next((f for f in all_files if f.suffix.lower() == ".edf"), None)
        annot_path = next((f for f in all_files if f.name.lower().endswith(".edf.xml")), None)

        if edf_path is None or annot_path is None:
            print(f"[WARNING] Missing EDF or annotation in {folder_name}")
            continue

        try:
            edf_file = pyedflib.EdfReader(str(edf_path))
            n_signals = edf_file.signals_in_file
            channel_names = edf_file.getSignalLabels()
            sample_freqs = {channel_names[i]: edf_file.getSampleFrequency(i) for i in range(n_signals)}
            duration_sec = edf_file.file_duration
            start_datetime = edf_file.getStartdatetime().isoformat()

            edf_file.close()
        except Exception as e:
            print(f"[ERROR] Cannot read {edf_path.name}: {e}")
            continue

        record = {
            "SubjectID": sid,
            "Site": site,
            "Dataset": dataset,
            "Visit": visit,
            "EDFPath": str(edf_path),
            "AnnotPath": str(annot_path),
            "RecordingStartTime": start_datetime,
            "RecordingDurationSec": duration_sec,
            "ChannelLabels": channel_names,
            "SamplingFreqs": sample_freqs,
        }

        records.append(record)

# Convert to DataFrame
df = pd.DataFrame(records)

# Save
output_path = Path("../datasets/mastersheets/mastersheet_MrOS_visit2.csv")
output_path.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(output_path, index=False)

print(f"✅ Saved mastersheet to {output_path}")

✅ Saved mastersheet to ../datasets/mastersheets/mastersheet_MrOS_visit2.csv
